---

# Proyecto 1: Hack a Boss
## Análisis del Top 25 ETFs

Proyecto de análisis del Top 25 de ETFs.

---





# Instalación e importación de **librerías**

In [ ]:
# Instalar las librerías
%pip install matplotlib
%pip install seaborn
%pip install yfinance
%pip install yahooquery
%pip install yahoofinancials
%pip install nbformat
%pip install scipy
%pip install plotly

In [3]:
# Importar las librerías
import time
import requests
import warnings
import numpy as np
import yfinance as yf
import yahooquery as yq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from bs4 import BeautifulSoup
from pprint import pprint
from tqdm import tqdm
from scipy import stats
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.fundamentaldata import FundamentalData
warnings.filterwarnings("ignore")

In [ ]:
ts = TimeSeries(output_format = 'pandas', indexing_type = 'date')
ti = TechIndicators(output_format = 'pandas', indexing_type = 'date')
fd = FundamentalData(output_format = 'pandas', indexing_type = 'date')

# Descarga y extracción de datos (API y WebScraping)





In [ ]:
HEADERS = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

def seleccion_etf(tipo_etfs:str, num:int) -> list:
    # Obtención de los símbolos del Top ETFs de EEUU
    s = yq.Screener()
    dict_query = s.get_screeners(tipo_etfs, num)
    tickers = [symbol['symbol'] for symbol in dict_query[tipo_etfs]['quotes']]
    return tickers

def corregir_nombres_columnas(df_columns:pd.Index) -> pd.Index:
    # Eliminar caracteres especiales
    df_columns = df_columns.str.replace('[^\w\s]', '', regex=True)
    # Sustituir espacios por _
    df_columns = df_columns.str.replace(' ', '_')
    # Retornar el DF
    return(df_columns)

def convertir_a_largo(df):
    # Resetear índice
    df_reset = df.reset_index()
    # Convertir el dataframe a formato largo
    df_melted = df_reset.melt(id_vars='Date', var_name=['Attribute', 'Ticker'], value_name='Value')
    # Una columna para cada cosa
    df_melted['Attribute'] = df_melted['Attribute'].astype(str)
    df_pivoted = df_melted.pivot_table(index=['Date', 'Ticker'], columns='Attribute', values='Value').reset_index()
    # Ponerlo todo bien y en orden
    df_pivoted.columns.name = None
    df_pivoted = df_pivoted[['Date', 'Ticker', 'Adj Close', 'Volume']]
    df_pivoted = df_pivoted.rename(
        columns={
            'Date':'Fecha',
            'Ticker':'Ticker',
            'Adj Close':'Precio',
            'Volume':'Volumen'
            }
        )
    # Retornar dataframe
    return df_pivoted

def convertir_a_ancho(df_largo):
    # Convertir el DataFrame a formato largo
    df_wide = df_largo.pivot_table(index='Fecha', columns='Ticker', values=['Precio', 'Volumen'], aggfunc='first')
    # Retornar el DataFrame en formato ancho
    return df_wide

def down_price_vol(tickers:list) -> pd.DataFrame:
    # Descarga de las cotizaciones
    df = yf.download(tickers, period='max')
    # Obtener los nombres de las columnas para 'Adj Close' y 'Volume'
    adj_close_cols = [('Adj Close', ticker) for ticker in tickers]
    volume_cols = [('Volume', ticker) for ticker in tickers]

    # Seleccionar las columnas de interés
    df = df[adj_close_cols + volume_cols]
    # Eliminar valores nulos y cambiar los precios por incrementos porcentuales
    df = df.dropna().pct_change(1).dropna()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    # Retornar DataFrame
    return(df)

def web_scraping_sectors(tickers:list) -> pd.DataFrame:
    url_base = "https://es.finance.yahoo.com/quote/"
    lista_sectores = [
        "Materiales básicos", "Acciones cíclicas", "Servicios financieros",
        "Propiedades inmobiliarias", "Acciones defensivas", "Atención sanitaria",
        "Utilidades", "Servicios de comunicación", "Energía", "Industriales",
        "Tecnología"
    ]
    dict_sectores = {}
    for etf in tqdm(tickers):
        url = f'{url_base}{etf}/holdings?p={etf}'
        response = requests.get(url, headers = HEADERS)
        soup = BeautifulSoup(response.text, 'html.parser')
        sectores_etf = {}
        lista_contenedores = soup.findAll('div', class_='Bdbw(1px) Bdbc($seperatorColor) Bdbs(s) H(25px) Pt(10px)')
        for item in lista_contenedores:
            sector = item.find('span', class_='Mend(5px) Whs(nw)').text
            if sector in lista_sectores:
                valor = item.find('span', class_='W(20%) D(b) Fl(start) Ta(e)').text
                sectores_etf[sector] = valor
        dict_sectores[etf] = sectores_etf
        time.sleep(0.5)
    # Convertir diccionario en dataframe
    df = pd.DataFrame.from_dict(dict_sectores).T
    # Convertir todas las entradas a strings, eliminar el '%' y reemplazar comas por puntos
    df = df.map(lambda x: str(x).replace('%', '').replace(',', '.'))
    # Convertir las cadenas a números flotantes y dividir por 100 para tener los porcentajes en formato decimal
    df = df.map(lambda x: float(x) / 100.0 if x != 'N/A' else 0)
    # Quitar los espacios
    df.columns = corregir_nombres_columnas(df.columns)
    # Resetear índice
    df = df.reset_index()
    df = df.rename(columns={'index':'Ticker'})
    # Retornar DataFrame
    return (df)

def tabla_net_assets(tickers:list) -> pd.DataFrame:
    def convertir_valor(valor):
        if 'M' in valor:
            return float(valor.replace('M', ''))
        elif 'B' in valor:
            return float(valor.replace('B', '')) * 1000
        else:
            return float(valor)
    #Net assets table
    net_assets = pd.DataFrame()
    for ticker in tqdm(tickers):
        url = f"https://finance.yahoo.com/quote/{ticker}?p={ticker}"
        response = requests.get(url, headers=HEADERS)
        try:
            tables = pd.read_html(response.text)
            for table in tables:
                labels_to_remove = ["Previous Close", "Open", "Bid", "Ask", "Day's Range", "52 Week Range", "Volume","Avg. Volume"]
                table = table[~table.iloc[:, 0].isin(labels_to_remove)]
                data_series = pd.Series(dict(zip(table.iloc[:, 0], table.iloc[:, 1])))
                net_assets[ticker] = data_series
        except ValueError:
            print(f"No hay tablas para {ticker}")
        time.sleep(0.5)
    net_assets = net_assets.T
    # Eliminar columnas
    df = net_assets.drop(['NAV', 'PE Ratio (TTM)', 'Expense Ratio (net)'], axis=1)
    # Corregir nombres de columnas
    df.columns = corregir_nombres_columnas(df.columns)
    # Convertir todas las entradas a strings, eliminar el '%' y reemplazar comas por puntos
    df[['Yield', 'YTD_Daily_Total_Return']] = df[['Yield', 'YTD_Daily_Total_Return']].map(lambda x: str(x).replace('%', ''))
    # Convertir las cadenas a números flotantes y dividir por 100 para tener los porcentajes en formato decimal
    df[['Yield', 'YTD_Daily_Total_Return']] = df[['Yield', 'YTD_Daily_Total_Return']].map(lambda x: float(x) / 100.0)
    # Convertir la columna Beta a float
    df['Beta_5Y_Monthly'] = df['Beta_5Y_Monthly'].astype(float)
    # Convertir Inception_Date en datetime
    df['Inception_Date'] = pd.to_datetime(df['Inception_Date'], format='%Y-%m-%d')
    # Reformatear columna Net_Assets
    df['Net_Assets_Millions'] = df['Net_Assets'].apply(convertir_valor)
    df = df.drop('Net_Assets', axis=1)
    # Resetear índice
    df = df.reset_index()
    df = df.rename(columns={'index':'Ticker'})
    # Retornar df
    return(df)

def portafolio_activos_etfs(tickers:list)->pd.DataFrame:
    all_dataframes = []
    for ticker in tqdm(tickers):
        url = f"https://finance.yahoo.com/quote/{ticker}/holdings?p={ticker}"
        response = requests.get(url, headers=HEADERS)
        try:
            tables = pd.read_html(response.text)
            for table in tables:
                table['Ticker'] = ticker  # Add the 'Ticker' column
                all_dataframes.append(table)
        except ValueError:
            print(f"No tables found for {ticker}")
        time.sleep(0.5)
    # Concatenate all dataframes
    df = pd.concat(all_dataframes, ignore_index=True)
    # Corregir nombres de columnas
    df.columns = corregir_nombres_columnas(df.columns)
    df.rename(columns={"_Assets":"Assets"}, inplace=True)
    # Convertir todas las entradas a strings, eliminar el '%' y reemplazar comas por puntos
    df['Assets'] = df['Assets'].map(lambda x: str(x).replace('%', ''))
    # Convertir las cadenas a números flotantes y dividir por 100 para tener los porcentajes en formato decimal
    df['Assets'] = df['Assets'].map(lambda x: float(x) / 100.0)
    # Retornar df
    return(df)

In [ ]:
# Selección ETFs
tickers = seleccion_etf(1000)

In [ ]:
# Descargar y extraer todos los datos
df_price_vol = convertir_a_largo(down_price_vol(tickers))
df_sect_dist = web_scraping_sectors(tickers)
df_net_activos = tabla_net_assets(tickers)
df_portafolio_activos = portafolio_activos_etfs(tickers)

In [ ]:
# Obtener el índice de referencia
df_price_vol_temp = convertir_a_ancho(df_price_vol)
sp500 = yf.download(tickers = '^GSPC', period='max')
sp500 = sp500['Adj Close'].dropna().pct_change(1).dropna()
sp500.index.name = 'Fecha'
sp500 = sp500.reindex(df_price_vol_temp.index)
# Se calcula la Beta y se asigna a la columna
for ticker in tickers:
    beta = df_price_vol_temp[('Precio', ticker)].cov(sp500) / sp500.var()
    df_net_activos.loc[df_net_activos['Ticker'] == ticker, 'Beta_5Y_Monthly'] = beta

# Visualizaciones Estadísticas

In [ ]:
### Descarga de datos desde Airtable ###
# Dataframe sectores
air = airtable.Airtable(base_id=base_id, table_name='tblWV2ERYBOrXozth', api_key=api_key)
lista_de_diccionarios = air.get_all()
lista_de_fields = [d['fields'] for d in lista_de_diccionarios]
df_divsf_sect = pd.DataFrame(lista_de_fields)
# Dataframe de activos netos
air = airtable.Airtable(base_id=base_id, table_name='tblvbY1n5rzmFyBqZ', api_key=api_key)
lista_de_diccionarios = air.get_all()
lista_de_fields = [d['fields'] for d in lista_de_diccionarios]
df_divsf_activos = pd.DataFrame(lista_de_fields)
df_divsf_activos_reducido = df_divsf_activos[['Ticker', 'Yield', 'Beta_5Y_Monthly']]
# Merge
df_divsf_rend = pd.merge(df_divsf_sect, df_divsf_activos_reducido, on='Ticker', how='right')
df_divsf_rend = df_divsf_rend.rename(columns={'Yield':'Rentabilidad', 'Beta_5Y_Monthly':'Riesgo'})

In [ ]:
### Identificar el sector con la mayor inversión en promedio ###
# Cálculo y selección
sector_columns = df_divsf_rend.columns[2:-3]
df_grafico_1 = df_divsf_rend[sector_columns].mean().sort_values(ascending=False)
# Visualización
plt.figure(figsize=(14,7))
df_grafico_1.plot(kind='bar', color='skyblue')
plt.title('Inversión Promedio en Cada Sector')
plt.xlabel('Sectores')
plt.ylabel('Inversión Promedio (Proporción)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
### Determinar el umbral para los ETFs más rentables (el cuartil superior) ###
quantil_rentabilidad = df_divsf_rend['Rentabilidad'].quantile(0.75)
etfs_mas_rentables = df_divsf_rend[df_divsf_rend['Rentabilidad'] >= quantil_rentabilidad]
# Calcular
df_grafico_2 = etfs_mas_rentables[sector_columns].mean().sort_values(ascending=False)
# Visualizar
plt.figure(figsize=(14,7))
df_grafico_2.plot(kind='bar', color='green')
plt.title('Inversión Promedio en Cada Sector para los ETFs más Rentables')
plt.xlabel('Sectores')
plt.ylabel('Inversión Promedio (Proporción)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
### Top rentabilidad-riesgo ETFs ###
plt.figure(figsize=(8, 6))
sns.set_style("whitegrid")
sns.scatterplot(data=df_divsf_activos, x="Beta_5Y_Monthly", y="Yield")
plt.title("Relación entre Riesgo (Beta) y Rentabilidad (Yield)")
plt.xlabel("Beta 5Y Monthly")
plt.ylabel("Yield")
plt.show()

In [ ]:
### Descarga de datos desde Airtable ###
air = airtable.Airtable(base_id=base_id, table_name='tblvxs74FGznyUCfL', api_key=api_key)
lista_de_diccionarios = air.get_all()
lista_de_fields = [d['fields'] for d in lista_de_diccionarios]
df_activos_repiten = pd.DataFrame(lista_de_fields)

In [ ]:
### Portafolio de activos recurrentes ###
# Repeticiones
symbol_repetitions = df_activos_repiten['Symbol'].value_counts().reset_index()
symbol_repetitions.columns = ['Symbol', 'Repetitions']
# Filtrar
symbol_repetitions = symbol_repetitions[symbol_repetitions['Repetitions'] > 1]
# Mediana
median_assets_by_symbol = df_activos_repiten[df_activos_repiten['Symbol'].isin(symbol_repetitions['Symbol'])]
median_assets_by_symbol = median_assets_by_symbol.groupby('Symbol')['Assets'].median().reset_index()
median_assets_by_symbol.columns = ['Symbol', 'Median_Assets']
# Obtener el nombre
name_by_symbol = df_activos_repiten.groupby('Symbol')['Name'].first().reset_index()
# Ordenar resultado
median_assets_by_symbol = median_assets_by_symbol.sort_values(by='Median_Assets', ascending=False)
# Visualizar
fig = px.bar(median_assets_by_symbol, x='Symbol', y='Median_Assets', color=symbol_repetitions['Repetitions'],
             color_continuous_scale=px.colors.sequential.Viridis,
             labels={'Symbol': 'Symbol', 'Median_Assets': 'Median Assets', 'color': 'Repeticiones'},
             title='Mediana de Assets por Symbol con Repeticiones (Excluyendo Repeticiones Únicas)')
fig.update_xaxes(tickangle=45)
fig.update_traces(text=name_by_symbol['Name'], textposition='outside')
fig.update_layout(xaxis_title='Symbol', yaxis_title='Median Assets')
fig.update_layout(coloraxis_showscale=False)
fig.show()

In [ ]:
### Descarga de datos ###
air = airtable.Airtable(base_id=base_id, table_name='tblkKLqcgl5dDo6uI', api_key=api_key)
lista_de_diccionarios = air.get_all()
lista_de_fields = [d['fields'] for d in lista_de_diccionarios]
df_price_vol_temp = convertir_a_ancho(pd.DataFrame(lista_de_fields))

In [ ]:
### Suma de los cambios porcentuales diarios del Top 25 ###
# Sumamos todas las variaciones porcentuales para obtener la variación total
total_pct_change = pd.DataFrame(df_price_vol_temp['Precio'].sum(axis=1))
# Visualizar
sns.histplot(total_pct_change, bins=100, kde=False, color="green")
# Distribución
params = stats.norm.fit(total_pct_change)
x = np.linspace(total_pct_change.min(), total_pct_change.max(), 100)
y = stats.norm.pdf(x, *params)
plt.plot(x, y, color="red") 
plt.show()

In [ ]:
df_capital = (df_price_vol_temp['Precio'] + 1).cumprod().sum(axis=1)
df_capital.plot()
plt.show()

# Para continuar con el trabajo ...

In [ ]:
annual_report = {
    "operatingCashflow": "10435000000",  # Efectivo de operaciones
    "capitalExpenditures": "1346000000",  # Gastos de capital
    "netIncome": "1639000000",  # Ingreso neto
    # ...Incluir el resto de los campos...
}

# Convertir todas las cadenas a enteros o floats según sea necesario
for key in annual_report:
    annual_report[key] = int(annual_report[key]) if annual_report[key] is not None else 0

# Free Cash Flow (FCF)
def calculate_fcf(operating_cash_flow, capital_expenditures):
    return operating_cash_flow - capital_expenditures

# Free Cash Flow Yield
def calculate_fcf_yield(free_cash_flow, market_cap):
    return free_cash_flow / market_cap

# Margen de FCF
def calculate_fcf_margin(free_cash_flow, total_revenue):
    return free_cash_flow / total_revenue

# Crecimiento del FCF YoY no puede calcularse solo con un año de datos
# Necesitaría al menos dos años de datos para calcular esto.

# Ratios de rentabilidad (como ROE, ROA, ROI)
def calculate_roi(net_income, total_investment):
    return net_income / total_investment

def calculate_roe(net_income, shareholder_equity):
    return net_income / shareholder_equity

def calculate_roa(net_income, total_assets):
    return net_income / total_assets

# Ratios de valoración (como P/E, P/FCF)
# Para estos cálculos necesitaríamos el precio de las acciones (P) y el número de acciones en circulación para calcular el P/E y P/FCF

# El Análisis de deuda requeriría datos sobre la deuda total y los intereses pagados

# Ratios de liquidez (como Current Ratio, Quick Ratio)
# Necesitaríamos conocer el total de activos corrientes y pasivos corrientes para calcular estos

# Efficiency Ratios (como rotación de inventario, DSO, y DPO)
# También se necesitarían datos adicionales, como ingresos por ventas, costo de bienes vendidos e inventario inicial y final
